In [ ]:
import sys; sys.path.append("../bazel-bin/src/py")

In [ ]:
import json
import numpy as np
import random
from tequila import data, spatial, voxels

In [ ]:
table = data.Table("octree_world")

In [ ]:
HEIGHT_MAP_SIZE = 512
VOXEL_ARRAY_SIZE = 64
GRID_SIZE = HEIGHT_MAP_SIZE // VOXEL_ARRAY_SIZE

In [ ]:
class VoxelInserter:
    
    def __init__(self, table):
        self.table = table
        self.voxel_arrays = {}
    
    def get(self, gx, gy, gz):
        vx = gx // VOXEL_ARRAY_SIZE
        vy = gy // VOXEL_ARRAY_SIZE
        vz = gz // VOXEL_ARRAY_SIZE
        ix = gx % VOXEL_ARRAY_SIZE
        iy = gy % VOXEL_ARRAY_SIZE
        iz = gz % VOXEL_ARRAY_SIZE
        if (vx, vy, vz) not in self.voxel_arrays:
            index = vx + vy * GRID_SIZE + vz * GRID_SIZE * GRID_SIZE
            self.voxel_arrays[(vx, vy, vz)] = voxels.loads(self.table.get(f"voxels/{index}"))
        return self.voxel_arrays[(vx, vy, vz)].get(ix, iy, iz)
        
    def set(self, gx, gy, gz, voxel):
        vx = gx // VOXEL_ARRAY_SIZE
        vy = gy // VOXEL_ARRAY_SIZE
        vz = gz // VOXEL_ARRAY_SIZE
        ix = gx % VOXEL_ARRAY_SIZE
        iy = gy % VOXEL_ARRAY_SIZE
        iz = gz % VOXEL_ARRAY_SIZE
        if (vx, vy, vz) not in self.voxel_arrays:
            index = vx + vy * GRID_SIZE + vz * GRID_SIZE * GRID_SIZE
            self.voxel_arrays[(vx, vy, vz)] = voxels.loads(self.table.get(f"voxels/{index}"))
        return self.voxel_arrays[(vx, vy, vz)].set(ix, iy, iz, voxel)
        
    def flush(self):
        for (vx, vy, vz), va in self.voxel_arrays.items():
            index = vx + vy * GRID_SIZE + vz * GRID_SIZE * GRID_SIZE
            self.table.set(f"voxels/{index}", voxels.dumps(va))
        self.voxel_arrays = {}

In [ ]:
def find_base_voxel(voxel_inserter, x, z):
    for y in range(511, -1, -1):
        voxel = voxel_inserter.get(x, y, z)
        if voxel:
            return y, voxel
    return None, None

In [ ]:
%%time

voxel_inserter = VoxelInserter(table) 
for tree_iteration in range(1000): 
    # 1. Pick a random coordinate, and find the base height.
    # 2. Make sure base voxel is grass.
    # 3. Randomly choose tree parameters.
    # 4. Test to see if tree will collide with anything.
    # 5. Assuming no collisions, insert tree.
    
    x = random.randint(0, 505)
    z = random.randint(0, 505)
    if x < 220 and z < 150:
        print("Failed: Bad bad starting coordinate.")
        continue
    
    # Find vy and the corresponding voxel array.
    y, voxel = find_base_voxel(voxel_inserter, x, z)
    if not voxel or voxel != 2 or y > 500:
        print("Failed: Bad base voxel.")
        continue      
        
    y += 1
    tree_height = random.randint(5, 9)
    tree_insertions = []
    
    # Define trunk.
    for i in range(tree_height):
        tree_insertions.append(
            (15, (x, y + i, z))
        )
        
    # Define leaf layer 0.
    tree_insertions.append((6, (x - 1, y + tree_height - 1, z - 1)))
    tree_insertions.append((6, (x + 1, y + tree_height - 1, z - 1)))
    tree_insertions.append((6, (x - 1, y + tree_height - 1, z + 1)))
    tree_insertions.append((6, (x + 1, y + tree_height - 1, z + 1)))
    
    # Define leaf layer 1.
    for tx, ty, tz in [
        (x + i, y + tree_height, z + j)
        for i in [-2, -1, 0, 1, 2]
        for j in [-2, -1, 0, 1, 2]
    ]:
        if abs(tx - x) == 2 and abs(tz - z) == 2:
            continue
        tree_insertions.append((6, (tx, ty, tz)))

    # Define leaf layer 2.
    for tx, ty, tz in [
        (x + i, y + tree_height + 1, z + j)
        for i in [-2, -1, 0, 1, 2]
        for j in [-2, -1, 0, 1, 2]
    ]:
        tree_insertions.append((6, (tx, ty, tz)))
                               
    # Define leaf layer 3.
    for tx, ty, tz in [
        (x + i, y + tree_height + 2, z + j)
        for i in [-2, -1, 0, 1, 2]
        for j in [-2, -1, 0, 1, 2]
    ]:
        if abs(tx - x) == 2 and abs(tz - z) == 2:
            continue
        tree_insertions.append((6, (tx, ty, tz)))

    # Define leaf layer 4.
    for tx, ty, tz in [
        (x + i, y + tree_height + 3, z + j)
        for i in [-1, 0, 1]
        for j in [-1, 0, 1]
    ]:
        tree_insertions.append((6, (tx, ty, tz)))

    # Check for collisions.
    if any([
        voxel_inserter.get(tx, ty, tz) != 0
        for _, (tx, ty, tz) in tree_insertions
    ]):
        print("Failed: Collision.")
        continue
                               
    # Yay, let's add our tree!
    for voxel_type, (tx, ty, tz) in tree_insertions:
        voxel_inserter.set(tx, ty, tz, voxel_type)
            
    voxel_inserter.flush()
    print(f"Success: coordinate {(x, y, z)}")